# Installation and Setup

The following steps are needed if you want to run the examples in this notebook on Google Colaboratory. If you want to run this notebook on your own machine, please follow the [installation instructions](https://camel-tools.readthedocs.io/en/latest/getting_started.html#installation) instead.

First, we install the CAMeL Tools Python package.

In [2]:
%pip install camel-tools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 KB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.6/426.6 KB 14.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 20.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.1 MB/s eta 0:00:00
  Created wheel for camel-kenlm: filename=camel_kenlm-2023.3.17.2-cp39-cp39-linux_x86_64.whl size=3543685 sha256=783e1550

In order to use all the components provided in CAMeL Tools, we need to install all the datasets required by these components.
To do this in Colab, we need to first mount a Google Drive and create a directory where the data will be installed.

Run the code below and follow the instructions in the output.

In [ ]:
# from google.colab import drive
# import os

# drive.mount('/gdrive')

# %mkdir /gdrive/MyDrive/camel_tools

Next, we need to tell CAMeL Tools to install the data in the newly created directory. This will take a couple of minutes to complete.

**NOTE:** You will need at least 2.3GB of available space on your Google Drive to install all the CAMeL Tools data.

In [ ]:
# os.environ['CAMELTOOLS_DATA'] = '/gdrive/MyDrive/camel_tools'

# !export | camel_data full

We also provide a lightweight dataset for the Morphology and Disambiguation components **only** that can be installed by calling `camel_data light` instead of `camel_data full`.


**Once the data has been installed on your Google Drive, you only need to run the following the next time you want to run this notebook.**

In [3]:
!camel_data -l

Package Name                      Size  License     Description
----------------------------  --------  ----------  -------------------------------------------------------------------------------------------------------------------------
all                                                 All available CAMeL Tools packages
defaults                                            Default datasets for all CAMeL Tools components
dialectid-all                                       All available dialect identification models
dialectid-model26             230.3 MB  MIT         Dialect identification model trained to differentiating between 25 Arabic city dialects as well as Modern Standard Arabic
dialectid-model6              117.7 MB  MIT         Dialect identification model trained to differentiating between 5 Arabic city dialects as well as Modern Standard Arabic
disambig-bert-unfactored-all                        All available unfactored BERT disambiguation models
disambig-bert-unfactored-egy

In [4]:
# !camel_data -i morphology-db-msa-r13

In [5]:
# %pip install camel-tools

from google.colab import drive
import os

drive.mount('/gdrive')
os.environ['CAMELTOOLS_DATA'] = '/gdrive/MyDrive/camel_tools'

Mounted at /gdrive


# Preprocessing

CAMeL Tools provides a suite of preprocessing utilities for cleaning and preparing Arabic text. Some of these may need to be used before passing text to other CAMeL Tools components.

In [6]:
from camel_tools.utils.charmap import CharMapper

sentence = "ذهبت إلى المكتبة."
print(sentence)

# http://languagelog.ldc.upenn.edu/myl/ldc/morph/buckwalter.html
ar2bw = CharMapper.builtin_mapper('ar2bw')

sent_bw = ar2bw(sentence)
print(sent_bw)

ذهبت إلى المكتبة.
*hbt <lY Almktbp.


`CharMapper` is a very flexible component that can be used for many other types of text transformations and can be initialized with any character-to-string mapping. 

### Unicode Normalization

Text in Python 3 is represented as a series of [Unicode](https://unicode.org/standard/standard.html) characters. Some characters have different variants, for example the character 'ع' can also be represented as 'ﻊ', 'ﻋ', 'ﻌ' depending on where it appears in a word. For all these forms, we would call 'ع' their canonical form. Furthermore, some Unicode characters are a composition of multiple characters and in some cases can comprise entire phrases. For example, the character 'ﰷ' is a single Unicode character that represents the composition of 'ك' and 'ا'. Unicode decomposition is the process of splitting these composed characters into their individual constituents.

These variants and composed forms are generally used for display purposes and are problematic in text processing tasks. To convert Unicode text into something more suitable for text processing, we provide the [`normalize_unicode`](https://camel-tools.readthedocs.io/en/latest/api/utils/normalize.html#camel_tools.utils.normalize.normalize_unicode) function.

In the example below, we demonstrate a more extreme example of how `normalize_unicode` converts a composed character into its decomposed form.


In [7]:
from camel_tools.utils.normalize import normalize_unicode

sentence = 'ﷺ'
print(sentence)

sent_norm = normalize_unicode(sentence)
print(sent_norm)

ﷺ
صلى الله عليه وسلم


**NOTE:** It is advised to run this function on all text prior to any further preprocessing or use.

### Orthographic Normalization

It is common for Arabic speakers to use shortcuts when typing Arabic text.
For example, the different variants of the letter alef ('ا', 'آ', 'أ', 'إ') may be typed as just 'ا'. Some of these substitutions can just be the result of typos. The presence of these variations can cause data sparsity and are usually normalized to a single form. Orthographic normalization is the process of converting letter variants or visually similar letters into a single form.

We provide a collection of orthographic normalization functions in CAMeL Tools. The example below demonstrates a few of them.

In [8]:
from camel_tools.utils.normalize import normalize_alef_maksura_ar
from camel_tools.utils.normalize import normalize_alef_ar
from camel_tools.utils.normalize import normalize_teh_marbuta_ar

sentence = "هل ذهبت إلى المكتبة؟"
print(sentence)

# Normalize alef variants to 'ا'
sent_norm = normalize_alef_ar(sentence)
print(sent_norm)

# Normalize alef maksura 'ى' to yeh 'ي'
sent_norm = normalize_alef_maksura_ar(sent_norm)
print(sent_norm)

# Normalize teh marbuta 'ة' to heh 'ه'
sent_norm = normalize_teh_marbuta_ar(sent_norm)
print(sent_norm)

هل ذهبت إلى المكتبة؟
هل ذهبت الى المكتبة؟
هل ذهبت الي المكتبة؟
هل ذهبت الي المكتبه؟


The example above performs orthographic normalization on Arabic script. We provide variants of these functions for other encoding schemes as well (e.g. Buckwalter, Safe Buckwalter, etc). See [here](https://camel-tools.readthedocs.io/en/latest/api/utils/normalize.html) for more information.


## Dediacritization

Dediacritization is the process of removing Arabic diacritical marks. Diacritics increase data sparsity and so most Arabic NLP techniques ignore them. The example below shows how diacritics can be removed from Arabic text using the [`dediac_ar`](https://camel-tools.readthedocs.io/en/latest/api/utils/dediac.html#camel_tools.utils.dediac.dediac_ar) function:

In [9]:
from camel_tools.utils.dediac import dediac_ar

sentence = "هَلْ ذَهَبْتَ إِلَى المَكْتَبَةِ؟"
print(sentence)

sent_dediac = dediac_ar(sentence)
print(sent_dediac)

هَلْ ذَهَبْتَ إِلَى المَكْتَبَةِ؟
هل ذهبت إلى المكتبة؟


We provide dediacritization functions for other encoding schemes (eg. Buckwalter, Safe Buckwalter, etc) which you can learn more about [here](https://camel-tools.readthedocs.io/en/latest/api/utils/dediac.html).


## Word Tokenization

Some CAMeL Tools components expect text to be pretokenized by whitespace and punctuation. This means that an input sentence should be an array of words and punctuation instead of a single string. This is to preserve the alignment of inputs and outputs.

Python does provide the `split()` method to tokenize words by whitespace but it doesn't seperate punctuation from words. For example:

In [10]:
sentence = "هَلْ ذَهَبْتَ إِلَى المَكْتَبَةِ؟"
print(sentence)

sent_split = sentence.split()
print(sent_split)

هَلْ ذَهَبْتَ إِلَى المَكْتَبَةِ؟
['هَلْ', 'ذَهَبْتَ', 'إِلَى', 'المَكْتَبَةِ؟']


To split by whitespace and seperate punctuation, we currently provide the utility function [`simple_word_tokenize`](https://camel-tools.readthedocs.io/en/latest/api/tokenizers/word.html#camel_tools.tokenizers.word.simple_word_tokenize). The example below is similar to the one above, but this time we use `simple_word_tokenize()` instead of `split()`.

In [11]:
from camel_tools.tokenizers.word import simple_word_tokenize

sentence = "هَلْ ذَهَبْتَ إِلَى المَكْتَبَةِ؟"
print(sentence)

sent_split = simple_word_tokenize(sentence)
print(sent_split)

هَلْ ذَهَبْتَ إِلَى المَكْتَبَةِ؟
['هَلْ', 'ذَهَبْتَ', 'إِلَى', 'المَكْتَبَةِ', '؟']


**NOTE:** `simple_word_tokenize`, as the name suggests is very simple and does not do anything fancy. It will seperate all punctuation regardless of context (ie. hashtags, emails, abbreviations, etc.)

# Morphology

CAMeL Tools provides a powerful morphological analysis, generation and reinflection system. This system is powered by [morphological databases](https://camel-tools.readthedocs.io/en/latest/api/morphology/database.html#databases) that are comprised of lexicons and compatibility tables. See [here](https://www.aclweb.org/anthology/W18-5816.pdf) for more details.

## Analysis


Morphological analysis is the process of generating all possible readings (analyses) of a given word out of context. All analyses are generated from the undiacritized form of the input word. Each of these analyses is defined by a set lexical and morphological features. 

In [12]:
from camel_tools.morphology.database import MorphologyDB
from camel_tools.morphology.analyzer import Analyzer


db = MorphologyDB.builtin_db('calima-msa-r13')
#https://camel-tools.readthedocs.io/en/latest/api/morphology/database.html
# db = MorphologyDB('/gdrive/MyDrive/camel_tools/data/morphology_db/calima-msa-r13/morphology.db')

analyzer = Analyzer(db)

analyses = analyzer.analyze('موظف')

for analysis in analyses:
    print(analysis, '\n')

{'diac': 'مُوَظَّف', 'lex': 'مُوَظَّف', 'bw': 'مُوَظَّف/ADJ', 'gloss': 'employed;hired', 'pos': 'adj', 'prc3': '0', 'prc2': '0', 'prc1': '0', 'prc0': '0', 'per': 'na', 'asp': 'na', 'vox': 'na', 'mod': 'na', 'stt': 'i', 'cas': 'u', 'enc0': '0', 'rat': 'n', 'source': 'lex', 'form_gen': 'm', 'form_num': 's', 'd3seg': 'مُوَظَّف', 'caphi': 'm_u_w_a_dh._dh._a_f', 'd1tok': 'مُوَظَّف', 'd2tok': 'مُوَظَّف', 'pos_logprob': -0.9868824, 'd3tok': 'مُوَظَّف', 'd2seg': 'مُوَظَّف', 'pos_lex_logprob': -5.400551, 'num': 's', 'ud': 'ADJ', 'gen': 'm', 'catib6': 'NOM', 'root': '#.ظ.ف', 'bwtok': 'مُوَظَّف', 'pattern': 'مُوَ2َّ3', 'lex_logprob': -5.400551, 'atbtok': 'مُوَظَّف', 'atbseg': 'مُوَظَّف', 'd1seg': 'مُوَظَّف', 'stem': 'مُوَظَّف', 'stemgloss': 'employed;hired', 'stemcat': 'N-ap'} 

{'diac': 'مُوَظَّفَ', 'lex': 'مُوَظَّف', 'bw': 'مُوَظَّف/ADJ+َ/CASE_DEF_ACC', 'gloss': 'employed;hired+[def.acc.]', 'pos': 'adj', 'prc3': '0', 'prc2': '0', 'prc1': '0', 'prc0': '0', 'per': 'na', 'asp': 'na', 'vox': 'na', 

Let's examine just one of the output analyses generated from another word ('وسيكتبونها') to get a better idea of the verious morphological and lexical features that we provide. We provide a [handy reference page](https://camel-tools.readthedocs.io/en/latest/reference/camel_morphology_features.html) that explains what these features are and what their values mean.

```
diac: وَسَيَكْتُبُونَها
lex: كَتَب-u_1
bw: وَ/CONJ+سَ/FUT_PART+يَ/IV3MP+كْتُب/IV+ُونَ/IVSUFF_SUBJ:MP_MOOD:I+ها/IVSUFF_DO:3FS
gloss: and_+_will_+_they_(people)+write+it;them;her
pos: verb
prc3: 0
prc2: wa_conj
prc1: sa_fut
prc0: 0
per: 3
asp: i
vox: a
mod: i
stt: na
cas: na
enc0: 3fs_dobj
rat: n
source: lex
form_gen: m
form_num: p
pattern: وَسَيَ1ْ2ُ3ُونَها
root: ك.ت.ب
catib6: PRT+PRT+VRB+NOM
ud: CONJ+AUX+VERB+PRON
d1seg: وَ+_سَيَكْتُبُونَها
d1tok: وَ+_سَيَكْتُبُونَها
atbseg: وَ+_سَ+_يَكْتُبُونَ_+ها
d3seg: وَ+_سَ+_يَكْتُبُونَ_+ها
d2seg: وَ+_سَ+_يَكْتُبُونَها
d2tok: وَ+_سَ+_يَكْتُبُونَها
atbtok: وَ+_سَ+_يَكْتُبُونَ_+ها
d3tok: وَ+_سَ+_يَكْتُبُونَ_+ها
bwtok: وَ+_سَ+_يَ+_كْتُب_+ُونَ_+ها
pos_lex_logprob: -3.648503
caphi: w_a_s_a_y_a_k_t_u_b_uu_n_a_h_aa
pos_logprob: -1.023208
gen: m
lex_logprob: -3.648503
num: p
stem: كْتُب
stemgloss: write
stemcat: IV
```

## Generation

Generation is the process of inflecting a lemma
for a set of morphological features. The example below generates all the possible inflections for the lemma 'مُوَظَّف' (employee) that are masculine plural nouns.

In [13]:
from camel_tools.morphology.database import MorphologyDB
from camel_tools.morphology.generator import Generator

# We need to indicate that the database we are loading will be
# used for generation.
db = MorphologyDB('/gdrive/MyDrive/camel_tools/data/morphology_db/calima-msa-r13/morphology.db', flags='g')

generator = Generator(db)

lemma = 'مُوَظَّف'
features = {
    'pos': 'noun',
    'gen': 'm',
    'num': 'p'
}

analyses = generator.generate(lemma, features)

# Extract and print unique diacritizations from generated analyses
for diac in set([a['diac'] for a in analyses]):
    print(diac)

مُوَظَّفِينَ
مُوَظَّفِي
مُوَظَّفُو
مُوَظَّفُونَ


**NOTE:** `'pos'` is the only *required* feature that needs to be specified.

## Reinflection

Reinflection is the process of converting a given word in any form to a different form (i.e. tense, gender, etc). The CAMeL Tools reinflector works similar to the generator except that the word doesn't have to be a lemma and it is not have to be restricted to a specific `'pos'`. The example below reinflects the word 'شوارع' (streets) into its dual form with a proclitic 'ب'.

In [14]:
from camel_tools.morphology.database import MorphologyDB
from camel_tools.morphology.reinflector import Reinflector

# We need to indicate that the database we are loading will be
# used for reinflection.

db = MorphologyDB('/gdrive/MyDrive/camel_tools/data/morphology_db/calima-msa-r13/morphology.db', flags='r')


reinflector = Reinflector(db)

word = 'شوارع'
features = {
    'num': 'd',
    'prc1': 'bi_prep'
}

analyses = reinflector.reinflect(word, features)

# Extract and print unique diacritizations from reinflected analyses
for diac in set([a['diac'] for a in analyses]):
    print(diac)

بِشارِعَيْنِ
بِشارِعَيْ


# Tagging

In [15]:
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.disambig.mle import MLEDisambiguator
from camel_tools.tagger.default import DefaultTagger

mle = MLEDisambiguator.pretrained()
tagger = DefaultTagger(mle, 'pos')

# The tagger expects pre-tokenized text
sentence = simple_word_tokenize('نجح بايدن في الانتخابات')

pos_tags = tagger.tag(sentence)

print(pos_tags)

['verb', 'noun_prop', 'prep', 'noun']


# Morphological Tokenization

In [16]:
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.disambig.mle import MLEDisambiguator
from camel_tools.tokenizers.morphological import MorphologicalTokenizer

# The tokenizer expects pre-tokenized text
sentence = simple_word_tokenize('فتنفست الصعداء')
print(sentence)

# Load a pretrained disambiguator to use with a tokenizer
mle = MLEDisambiguator.pretrained('calima-msa-r13')

# By specifying `split=True`, the morphological tokens are output as seperate
# strings.
tokenizer = MorphologicalTokenizer(mle, scheme='d3tok', split=True)
tokens = tokenizer.tokenize(sentence)
print(tokens)

# We can output diacritized tokens by setting `diac=True`
tokenizer = MorphologicalTokenizer(mle, scheme='d3tok', split=True, diac=True)
tokens = tokenizer.tokenize(sentence)
print(tokens)

['فتنفست', 'الصعداء']
['ف+', 'تنفست', 'ال+', 'صعداء']
['فَ+', 'تَنَفَّسَت', 'ال+', 'صُعَداءَ']


# Dialect Identification

In [17]:
from camel_tools.dialectid import DialectIdentifier

import warnings
warnings.filterwarnings("ignore")

did = DialectIdentifier.pretrained()

sentences = [
    'مال الهوى و مالي شكون اللي جابني ليك  ما كنت انايا ف حالي بلاو قلبي يانا بيك',
    'بدي دوب قلي قلي بجنون بحبك انا مجنون ما بنسى حبك يوم'
]

predictions = did.predict(sentences, 'city')
print([p.top for p in predictions])

predictions = did.predict(sentences, 'country')
print([p.top for p in predictions])

predictions = did.predict(sentences, 'region')
print([p.top for p in predictions])

['Rabat', 'Aleppo']
['Morocco', 'Syria']
['Maghreb', 'Levant']


# Sentiment Analysis

In [18]:
from camel_tools.sentiment import SentimentAnalyzer

sa = SentimentAnalyzer.pretrained()

sentences = [
    'أنا بخير',
    'أنا لست بخير'
]

sentiments = sa.predict(sentences)

print(sentiments)

['positive', 'negative']


# Named Entitiy Recognition

CAMeL Tools comes with an easy-to-use, pretrained [named-entitity recognition](https://en.wikipedia.org/wiki/Named-entity_recognition)  (NER) system that can be accessed using the [`NERecognizer`](https://camel-tools.readthedocs.io/en/latest/api/ner.html#camel_tools.ner.NERecognizer) class. 

For each token in an input sentence, `NERecognizer` outputs a label that indicates the type of named-entity.The system outputs one of the following labels for each token: `'B-LOC'`, `'B-ORG'`, `'B-PERS'`, `'B-MISC'`, `'I-LOC'`, `'I-ORG'`, `'I-PERS'`, `'I-MISC'`, `'O'`.
Named-entites can either be a `LOC` (location), `ORG` (organization), `PERS` (person), or `MISC` (miscallaneous).

Labels beginning with `B` indicate that their corresponding tokens are the begininging of a multi-word named-entity or is a single-token named-entity'. Those begining with `I` indicate that their corresponding tokens are continuations of a multi-word named-entity. Words that aren't named-entities are given the `'O'` label.

The example below illustrates how `NERecognizer` can be used to label named-entities in a given sentence.


In [19]:
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.ner import NERecognizer

ner = NERecognizer.pretrained()

# NERecognizer expects pre-tokenized text
sentence = simple_word_tokenize('إمارة أبوظبي هي إحدى إمارات دولة الإمارات العربية المتحدة السبع.')

labels = ner.predict_sentence(sentence)

# Print each token paired with it's NER label
print(list(zip(sentence, labels)))

[('إمارة', 'O'), ('أبوظبي', 'B-LOC'), ('هي', 'O'), ('إحدى', 'O'), ('إمارات', 'O'), ('دولة', 'O'), ('الإمارات', 'B-LOC'), ('العربية', 'I-LOC'), ('المتحدة', 'I-LOC'), ('السبع', 'O'), ('.', 'O')]
